Visualizing DAO (1KSI)

In [56]:
import py3Dmol
py3Dmol.__version__
p = py3Dmol.view(query='pdb:1ksi')
p.setStyle({'cartoon': {'color':'spectrum'}})


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [57]:
p.setHoverable({},True,'''function(atom,viewer,event,container) {
                   if(!atom.label) {
                    atom.label = viewer.addLabel(atom.resn+":"+atom.atom,{position: atom, backgroundColor: 'mintcream', fontColor:'black'});
                   }}''',
               '''function(atom,viewer) { 
                   if(atom.label) {
                    viewer.removeLabel(atom.label);
                    delete atom.label;
                   }
                }''')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [58]:
p.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}})

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# Obtain trehalose and manipulate the molecule

Function to obtain CID code from pubchem. Also function to download the file
Adapted from Faris Izzatur Rahman [code](https://github.com/FarisIzzaturRahman/pubchem_3d_CSVdownloader/blob/main/README.md).

In [59]:
import pubchempy as pcp
import requests
import argparse

def search_pubchem(compound: str) -> int:
    """Search PubChem for a given compound and return its CID."""
    try:
        result = pcp.get_compounds(compound, 'name', record_type='3d')[0]
        return result.cid
    except (IndexError, pcp.PubChemHTTPError) as e:
        print(f"Error searching PubChem for {compound}: {e}")
        return None

def download_sdf(cid: int) -> str:
    """Download the 3D structure of a compound in SDF format from PubChem and return the SDF text."""
    try:
        url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF'
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except (requests.exceptions.RequestException, pcp.PubChemHTTPError) as e:
        print(f"Error downloading SDF for CID {cid}: {e}")
        return None

obtain the CID of trehalose

In [60]:
compound_name = 'trehalose'
cid = search_pubchem(compound_name)
sdf = download_sdf(cid)
try:
    with open(f'{compound_name}.sdf', 'w') as f:
        f.write(sdf)
    print(f"Downloaded SDF for {compound_name} with CID:{cid}")
except IOError as e:
    print(f"Error saving SDF for {compound_name}: {e}")

Downloaded SDF for trehalose with CID:7427


In [61]:
view = py3Dmol.view(width=680,height=250, query='cid:'+str(cid), viewergrid=(1,3), linked=True)

view.setStyle({'line': {'linewidth':8}}, viewer=(0,0))
view.setStyle({'stick': {'colorscheme':'cyanCarbon'}}, viewer=(0,1))
view.setStyle({'sphere': {}}, viewer=(0,2))

view.render()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

convert SDF to PDB (adapted from [rdkit-scripts](https://github.com/DrrDom/rdkit-scripts/blob/master/mols2pdb.py))

In [62]:
from rdkit import Chem
from rdkit.Chem import AllChem
from read_input import read_input

def convert_to_pdb(input):
    for mol, mol_name in read_input(input):
        print(mol, mol_name)
        if mol.GetNumConformers() == 0 or not mol.GetConformer(0).Is3D():
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol)
        else:
            mol = Chem.AddHs(mol, addCoords=True)
        mol.SetProp('_Name', mol_name)
        print(mol_name)
        Chem.MolToPDBFile(mol, mol_name + '.pdb', flavor=4)
   
    return 'Done'
convert_to_pdb('trehalose.sdf')
#convert_to_pdb(f'{compound_name}.sdf','./')

m = py3Dmol.view('7427.pdb',style='stick')
m.addStyle({'resn':'UNL'},{'stick':{'colorscheme':'grayCarbon'}})
m.render()

<rdkit.Chem.PropertyMol.PropertyMol object at 0x116681030> 7427
7427


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now use the `sed` command to modify the name of the residue in trehalose from `UNL` into `TRE`

In [64]:
import subprocess
subprocess.call(["sed -e 's/UNL/TRE/g' 7427.pdb >trehalose.pdb"], shell=True)

0